In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.1 MB/s eta 0:00:00


In [ ]:
import os
import torch
from torch.utils.data import Dataset, random_split, DataLoader, ConcatDataset, IterableDataset
import pytorch_lightning as pl
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models
import PIL
from tqdm import tqdm
import cv2
import copy
from sklearn.metrics import classification_report

In [ ]:
def transform_mnist(img):
  img = img.resize((32,32))
  tensor = transforms.ToTensor()(img)
  tensor = torch.concatenate([tensor,tensor,tensor],dim=0)
  return tensor

def transform_cmnist(img):
  img = img.resize((32,32))
  tensor = transforms.ToTensor()(img)
  tensor = torch.concatenate([tensor,tensor,tensor],dim=0) + torch.rand(size=(3,1,1))
  return tensor

def transform_svhn(img):
  img = img.resize((32,32))
  tensor = transforms.ToTensor()(img)
  return tensor

def confusion_matrix(pred,true):
  conf_matrix = np.zeros((10,10),dtype='uint8')
  for i in range(len(pred)):
    conf_matrix[pred[i]][true[i]]+=1
  print(conf_matrix)

def accuracy(pred,true):
  class_vector = np.zeros((1,10))
  total_vector = np.zeros((1,10))
  total_correct = 0
  for i in range(len(pred)):
    if pred[i]==true[i]:
      total_correct+=1
      class_vector[0][true[i]]+=1
    total_vector[0][true[i]]+=1
  print("\nOverall Accuracy = ", 100*total_correct/len(pred), "%")
  print("\nClass-Wise Accuracies = ", 100*class_vector/total_vector)
  print(classification_report(pred,true))

def detect(test_loader, model_class):
  pred=[]
  true=[]
  model_class.cuda()
  for i,j in tqdm(test_loader):
    i,j = i.cuda(), j.cuda()
    prob = model_class(i)
    out = prob.max(1, keepdim=True)[1]
    pred.append(out.detach().cpu().item())
    true.append(j.detach().cpu().item())
  print("Confusion Matrix:\n")
  confusion_matrix(pred, true)
  accuracy(pred, true)
  del pred, true

def federated_learning(datasets, num_epochs, name):
  num_clients = len(datasets)
  client_models=[]
  for i, dataset in enumerate(datasets):
    train_loader, test_loader = dataset
    client_model = Model(name,10)
    trainer = pl.Trainer(accelerator='gpu', max_epochs=num_epochs, default_root_dir='/content/drive/MyDrive/DAI_Assignment_3/'+str(name)+'_'+str(i+1), benchmark=True)
    trainer.fit(client_model, train_loader)
    client_models.append(client_model)
    del client_model, trainer
  model = Model(name,10)
  for name, param in model.named_parameters():
    param.data = torch.zeros_like(param.data)
    for client_model in client_models:
      param.data += client_model.state_dict()[name] / num_clients
  return model, client_models

def model_return(ckpt_path, model_obj):
  ckpt = torch.load(ckpt_path)
  model_obj.load_state_dict(ckpt['state_dict'])
  return model_obj.eval().cuda()

def random_pick_mnist(num_per_class, dataset, b_size):
  classes = list(range(10))
  indices = []
  for c in classes:
    indices.extend(torch.where(dataset.targets == c)[0][:num_per_class])
  subset_set = torch.utils.data.Subset(dataset, indices)
  loader = DataLoader(subset_set, batch_size=b_size, shuffle=True)
  return loader

def random_pick_svhn(num_per_class, dataset, b_size):
  classes = list(range(10))
  indices = []
  for c in classes:
    indices.extend(np.where(dataset.labels == c)[0][:num_per_class])
  subset_set = torch.utils.data.Subset(dataset, indices)
  loader = DataLoader(subset_set, batch_size=b_size, shuffle=True)
  return loader

def server_aggregate(global_model, client_models):
  global_dict = global_model.state_dict()
  for k in global_dict.keys():
    global_dict[k] = torch.stack([client_models[i].state_dict()[k].float() for i in range(len(client_models))], 0).mean(0)
  global_model.load_state_dict(global_dict)
  for model in client_models:
    model.load_state_dict(global_model.state_dict())
  return global_model

In [ ]:
class Model(pl.LightningModule):

  def __init__(self, model_name, num_classes):
    super().__init__()
    self.model_name = model_name
    if self.model_name=='resnet_34':
      self.model = models.resnet34(weights=None)
      self.features = self.model.fc.in_features 
      self.model.fc = nn.Linear(self.features,num_classes)
    if self.model_name=='convnext_tiny':
      self.model = models.convnext_tiny(weights=None)
      self.features = self.model.classifier[-1].in_features 
      self.model.classifier[-1] = nn.Linear(self.features,num_classes)
    if self.model_name=='googlenet':
      self.model = models.googlenet(weights=None)
      self.features = self.model.fc.in_features 
      self.model.fc = nn.Linear(self.features,num_classes)
    if self.model_name=='mnasnet0_5':
      self.model = models.mnasnet0_5(weights=None)
      self.features = self.model.classifier[-1].in_features 
      self.model.classifier[-1] = nn.Linear(self.features,num_classes)
    if self.model_name=='regnet_y_400mf':
      self.model = models.regnet_y_400mf(weights=None)
      self.features = self.model.fc.in_features 
      self.model.fc = nn.Linear(self.features,num_classes)
    if self.model_name=='resnext50_32x4d':
      self.model = models.resnext50_32x4d(weights=None)
      self.features = self.model.fc.in_features 
      self.model.fc = nn.Linear(self.features,num_classes)
    if self.model_name=='shufflenet_v2_x0_5':
      self.model = models.shufflenet_v2_x0_5(weights=None)
      self.features = self.model.fc.in_features 
      self.model.fc = nn.Linear(self.features,num_classes)
    if self.model_name=='squeezenet1_0':
      self.model = models.squeezenet1_0(weights=None)
      self.features = self.model.classifier[-3].in_channels
      self.model.classifier[-3] = nn.Conv2d(self.features, num_classes, kernel_size=(1, 1), stride=(1, 1))
    if self.model_name=='vgg11':
      self.model = models.vgg11(weights=None)
      self.features = self.model.classifier[-1].in_features 
      self.model.classifier[-1] = nn.Linear(self.features,num_classes)
    if self.model_name=='wide_resnet50_2':
      self.model = models.wide_resnet50_2(weights=None)
      self.features = self.model.fc.in_features 
      self.model.fc = nn.Linear(self.features,num_classes)
    if self.model_name=='swin_t':
      self.model = models.swin_t(weights=None)
      self.features = self.model.head.in_features
      self.model.head = nn.Linear(self.features,num_classes)
    if self.model_name=='densenet_121':
      self.model = models.densenet121(weights=None)
      self.features = self.model.classifier.in_features
      self.model.classifier = nn.Linear(self.features,num_classes)
    if self.model_name=='efficientnet_b0':
      self.model = models.efficientnet.efficientnet_b0(weights=None)
      self.features = self.model.classifier[-1].in_features
      self.model.classifier[-1] = nn.Linear(self.features,num_classes)
    if self.model_name=='mobilenet_v3_large':
      self.model = models.mobilenet_v3_large(weights=None)
      self.features = self.model.classifier[-1].in_features
      self.model.classifier[-1] = nn.Linear(self.features,num_classes)
    self.loss = nn.CrossEntropyLoss()

  def forward(self, x):
    return self.model(x)

  def training_step(self, batch, batch_no):
    x, y = batch
    if self.model_name=='googlenet':
      logits = self(x).logits
    else:
      logits = self(x)
    loss = self.loss(logits, y)
    return loss

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(),lr=0.001)

In [ ]:
mnist_train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform_mnist)
mnist_test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform_mnist)
cmnist_train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform_cmnist)
cmnist_test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform_cmnist)
svhn_train_dataset = datasets.SVHN(root="./data", split="train", download=True, transform=transform_svhn)
svhn_test_dataset = datasets.SVHN(root="./data", split="test", download=True, transform=transform_svhn)

100%|██████████| 9912422/9912422 [00:00<00:00, 90119654.41it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 115038645.61it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 28189468.31it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 17655726.38it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 182040794/182040794 [00:04<00:00, 43944887.97it/s]


100%|██████████| 64275384/64275384 [00:02<00:00, 28282465.79it/s]


In [ ]:
mnist_train_loader = random_pick_mnist(1000, mnist_train_dataset,4)
mnist_test_loader = random_pick_mnist(500, mnist_test_dataset,1)
cmnist_train_loader = random_pick_mnist(1000, cmnist_train_dataset,4)
cmnist_test_loader = random_pick_mnist(500, cmnist_test_dataset,1)
svhn_train_loader = random_pick_svhn(1000, svhn_train_dataset,4)
svhn_test_loader = random_pick_svhn(500, svhn_test_dataset,1)
client_datasets = [(mnist_train_loader,mnist_test_loader),(cmnist_train_loader,cmnist_test_loader),(svhn_train_loader,svhn_test_loader)]

In [ ]:
train_datasets = [mnist_train_loader.dataset, cmnist_train_loader.dataset, svhn_train_loader.dataset]
test_datasets = [mnist_test_loader.dataset, cmnist_test_loader.dataset, svhn_test_loader.dataset]

train_dataset = ConcatDataset(train_datasets)
test_dataset = ConcatDataset(test_datasets)

class IterableConcatDataset(IterableDataset):
    def __init__(self, datasets):
        self.datasets = datasets

    def __iter__(self):
        for dataset in self.datasets:
            yield from iter(dataset)

train_iterable_dataset = IterableConcatDataset(train_datasets)
test_iterable_dataset = IterableConcatDataset(test_datasets)

train_dataloader = DataLoader(train_iterable_dataset, batch_size=4)
test_dataloader = DataLoader(test_iterable_dataset, batch_size=1)

# Using For-Loop Method

##GoogleNet

In [ ]:
model, tmodels = federated_learning(client_datasets, num_epochs=20, name='googlenet')

/usr/local/lib/python3.10/dist-packages/torchvision/models/googlenet.py:47: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | GoogLeNet        | 12.0 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
1

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
/usr/local/lib/python3.10/dist-packages/torchvision/models/googlenet.py:47: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | GoogLeNet        |

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | GoogLeNet        | 12.0 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
12.0 M    Trainable params
0         Non-trainable params
12.0 M    Total params
47.961    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
  0%|          | 0/5000 [00:00<?, ?it/s]


ValueError: ignored

In [ ]:
detect(mnist_test_loader,model.eval())
detect(cmnist_test_loader,model.eval())
detect(svhn_test_loader,model.eval())

100%|██████████| 5000/5000 [00:50<00:00, 99.64it/s] 
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0. 100.   0.   0.   0.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.10      0.18      5000
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

100%|██████████| 5000/5000 [00:50<00:00, 98.81it/s] 
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0. 100.   0.   0.   0.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.10      0.18      5000
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

100%|██████████| 5000/5000 [00:45<00:00, 108.86it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0. 100.   0.   0.   0.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.10      0.18      5000
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
detect(mnist_test_loader,tmodels[0].eval())
detect(cmnist_test_loader,tmodels[1].eval())
detect(svhn_test_loader,tmodels[2].eval())
del model, tmodels

100%|██████████| 5000/5000 [00:48<00:00, 103.95it/s]


Confusion Matrix:

[[217   0   0   0   0   0   3   0   0   0]
 [  0 237   4   0   2   0   3  11   1   6]
 [  0   3 215   1   1   0   0  17   3   1]
 [  0   0   0 207   0   9   0   0   1   4]
 [  0   0   1   0 233   0   2   0   0  11]
 [ 18   0   0   3   0 206   8   0   4   1]
 [  2   0   0   0   0   0 164   0   0   0]
 [  2   0   1   3   1   3   0 207   4  13]
 [  5   4  23  29   2  26  64   5 230   4]
 [  0   0   0   1   5   0   0   4   1 204]]

Overall Accuracy =  93.6 %

Class-Wise Accuracies =  [[94.6 98.6 94.2 92.6 97.8 92.4 84.  92.6 97.2 92. ]]
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       476
           1       0.99      0.95      0.97       520
           2       0.94      0.95      0.94       497
           3       0.93      0.97      0.95       477
           4       0.98      0.97      0.98       503
           5       0.92      0.93      0.93       496
           6       0.84      1.00      0.91       422
         

100%|██████████| 5000/5000 [00:46<00:00, 106.45it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [ 80  58  67  87  73  86  76  69  78  70]
 [  0   0   0   0   0   0   0   0   0   0]
 [164 186 177 157 171 158 168 175 166 174]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  9.42 %

Class-Wise Accuracies =  [[ 0.  11.6  0.  82.6  0.   0.   0.   0.   0.   0. ]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.12      0.08      0.09       744
           2       0.00      0.00      0.00         0
           3       0.83      0.10      0.17      4256
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

100%|██████████| 5000/5000 [00:49<00:00, 101.56it/s]


Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [ 84  88  76  67  68  75  91  80  73  78]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [160 156 168 177 176 169 153 164 171 166]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.4 %

Class-Wise Accuracies =  [[ 0.  17.6  0.   0.  86.4  0.   0.   0.   0.   0. ]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.18      0.11      0.14       780
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.86      0.10      0.18      4220
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## MobileNet_v3_Large

In [ ]:
model, tmodels = federated_learning(client_datasets, num_epochs=20, name='mobilenet_v3_large')
detect(mnist_test_loader,model.eval())
detect(cmnist_test_loader,model.eval())
detect(svhn_test_loader,model.eval())

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | MobileNetV3      | 4.2 M 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
4.2 M     Trainable params
0         Non-trainable params
4.2 M     Total params
16.859    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | MobileNetV3      | 4.2 M 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
4.2 M     Trainable params
0         Non-trainable params
4.2 M     Total params
16.859    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | MobileNetV3      | 4.2 M 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
4.2 M     Trainable params
0         Non-trainable params
4.2 M     Total params
16.859    Total estimated model params size (MB)


...

100%|██████████| 5000/5000 [00:47<00:00, 105.26it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0. 100.   0.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       1.00      0.10      0.18      5000
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

100%|██████████| 5000/5000 [00:47<00:00, 105.24it/s]

Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0. 100.   0.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       1.00      0.10      0.18      5000
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
detect(mnist_test_loader,tmodels[0].eval())
detect(cmnist_test_loader,tmodels[1].eval())
detect(svhn_test_loader,tmodels[2].eval())
del model, tmodels

100%|██████████| 5000/5000 [00:48<00:00, 103.66it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0.   0.   0.   0. 100.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

100%|██████████| 5000/5000 [00:48<00:00, 103.96it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0.   0.   0.   0. 100.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

100%|██████████| 5000/5000 [00:46<00:00, 107.20it/s]

Confusion Matrix:

[[  1   0   1   1   1   0   0   2   2   1]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [232 226 227 227 229 224 229 227 230 234]
 [  3   4   6   1   3   6   4   1   5   3]
 [  8  14  10  15  11  14  11  14   7   6]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  9.98 %

Class-Wise Accuracies =  [[ 0.2  0.   0.   0.   0.  96.   0.8  2.8  0.   0. ]]
              precision    recall  f1-score   support

           0       0.00      0.11      0.00         9
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.96      0.10      0.18      4845
           6       0.01      0.11      0.01        36
         


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## SqueezeNet1_0

In [ ]:
model, tmodels = federated_learning(client_datasets, num_epochs=20, name='squeezenet1_0')
detect(mnist_test_loader,model.eval())
detect(cmnist_test_loader,model.eval())
detect(svhn_test_loader,model.eval())

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | SqueezeNet       | 740 K 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
740 K     Trainable params
0         Non-trainable params
740 K     Total params
2.962     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | SqueezeNet       | 740 K 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
740 K     Trainable params
0         Non-trainable params
740 K     Total params
2.962     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | SqueezeNet       | 740 K 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
740 K     Trainable params
0         Non-trainable params
740 K     Total params
2.962     Total estimated model params size (MB)


...

100%|██████████| 5000/5000 [00:17<00:00, 280.64it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:

[[244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[100.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       1.00      0.10      0.18      5000
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

100%|██████████| 5000/5000 [00:17<00:00, 291.89it/s]


Confusion Matrix:

[[244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[100.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       1.00      0.10      0.18      5000
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
detect(mnist_test_loader,tmodels[0].eval())
detect(cmnist_test_loader,tmodels[1].eval())
detect(svhn_test_loader,tmodels[2].eval())
del model, tmodels

100%|██████████| 5000/5000 [00:17<00:00, 287.75it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:

[[244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[100.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       1.00      0.10      0.18      5000
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

100%|██████████| 5000/5000 [00:18<00:00, 275.11it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:

[[244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[100.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       1.00      0.10      0.18      5000
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

100%|██████████| 5000/5000 [00:18<00:00, 270.22it/s]

Confusion Matrix:

[[244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[100.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       1.00      0.10      0.18      5000
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Shufflenet_v2_x0_5

In [ ]:
model, tmodels = federated_learning(client_datasets, num_epochs=20, name='shufflenet_v2_x0_5')
detect(mnist_test_loader,model.eval())
detect(cmnist_test_loader,model.eval())
detect(svhn_test_loader,model.eval())

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ShuffleNetV2     | 352 K 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
352 K     Trainable params
0         Non-trainable params
352 K     Total params
1.408     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ShuffleNetV2     | 352 K 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
352 K     Trainable params
0         Non-trainable params
352 K     Total params
1.408     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ShuffleNetV2     | 352 K 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
352 K     Trainable params
0         Non-trainable params
352 K     Total params
1.408     Total estimated model params size (MB)


...

100%|██████████| 5000/5000 [00:48<00:00, 104.04it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   5   1   0   0   2   0   2   0   0]
 [  0   5   0   0   0   0   2   3   0   0]
 [244 234 243 244 244 242 242 239 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.02 %

Class-Wise Accuracies =  [[  0.    0.    0.2   0.  100.    0.    0.    0.    0.    0. ]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.10      0.00        10
           3       0.00      0.00      0.00        10
           4       1.00      0.10      0.18      4980
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         

100%|██████████| 5000/5000 [00:46<00:00, 107.43it/s]

Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  9  18  12  18  19  12   5  15  13   5]
 [ 23  19   9  12   6  23  15  34   7  14]
 [212 207 223 214 219 209 224 195 224 225]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  9.98 %

Class-Wise Accuracies =  [[ 0.   0.   2.4  2.4 95.   0.   0.   0.   0.   0. ]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.02      0.10      0.04       126
           3       0.02      0.07      0.04       162
           4       0.95      0.10      0.18      4712
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
detect(mnist_test_loader,tmodels[0].eval())
detect(cmnist_test_loader,tmodels[1].eval())
detect(svhn_test_loader,tmodels[2].eval())
del model, tmodels

100%|██████████| 5000/5000 [00:47<00:00, 104.65it/s]


Confusion Matrix:

[[227   1  53   1  38 240 197  13  12  10]
 [  1 233   3   0  12   1   5   2   6   4]
 [  0   6 118   3   3   1   0   5  20   1]
 [  0   0   3 228   0   9   1   3  14   8]
 [  1   0   1   1 177   0   4   4   2  78]
 [  6   1   0   5   0 239  13   0   6   7]
 [  7   1   7   1   4   3  13   0   6   1]
 [  1   2  59   4   4   3   0 215  12  20]
 [  1   0   0   0   4   4  11   1 163   2]
 [  0   0   0   1   2   0   0   1   3 113]]

Overall Accuracy =  80.6 %

Class-Wise Accuracies =  [[96.6 97.8 74.8 96.8 86.6 47.8 53.8 94.2 83.8 73.8]]
              precision    recall  f1-score   support

           0       0.97      0.46      0.62      1048
           1       0.98      0.93      0.96       523
           2       0.75      0.91      0.82       413
           3       0.97      0.93      0.95       522
           4       0.87      0.83      0.85       524
           5       0.48      0.86      0.62       277
           6       0.54      0.90      0.67       299
         

100%|██████████| 5000/5000 [00:48<00:00, 103.70it/s]


Confusion Matrix:

[[234   2   5   3   5   3  22   0  14   5]
 [  0 237   4   6   1   0   3   3   8   2]
 [  0   5 190   6   1   0   4  11   1   0]
 [  0   0   3 181   0  17   0   3   1   3]
 [  1   0   1   0 199   0  12   2   1   7]
 [  1   0   0  19   0 184   1   0   2   2]
 [  1   0   7   4   7   3 193   0   1   0]
 [  1   0  26  11   2   4   0 215   3  11]
 [  3   0   7   6   2  25   8   3 202   9]
 [  3   0   1   8  27   8   1   7  11 205]]

Overall Accuracy =  92.0 %

Class-Wise Accuracies =  [[98.  98.6 89.2 87.4 91.  88.  89.8 94.2 91.6 92.2]]
              precision    recall  f1-score   support

           0       0.98      0.89      0.93       549
           1       0.99      0.95      0.97       520
           2       0.89      0.94      0.92       474
           3       0.87      0.94      0.91       464
           4       0.91      0.95      0.93       479
           5       0.88      0.95      0.91       465
           6       0.90      0.95      0.92       472
         

100%|██████████| 5000/5000 [00:46<00:00, 107.26it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:

[[  0   0   0   0   0   0   0   1   1   0]
 [ 57  59  54  48  62  60  62  72  56  59]
 [  0   0   0   0   0   0   0   0   0   0]
 [ 14   9  13   8   5  10   8  13   6  10]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  2   2   0   4   1   0   1   1   1   0]
 [ 97 109 109 106  91 107 100  85 109  86]
 [ 74  65  68  78  85  67  73  72  71  89]]

Overall Accuracy =  10.44 %

Class-Wise Accuracies =  [[ 0.  11.8  0.   1.6  0.   0.   0.   0.2 73.  17.8]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.12      0.10      0.11       589
           2       0.00      0.00      0.00         0
           3       0.02      0.08      0.03        96
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
        

## Convnext_tiny

In [ ]:
model, tmodels = federated_learning(client_datasets, num_epochs=20, name='convnext_tiny')
detect(mnist_test_loader,model.eval())
detect(cmnist_test_loader,model.eval())
detect(svhn_test_loader,model.eval())

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ConvNeXt         | 27.8 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
27.8 M    Trainable params
0         Non-trainable params
27.8 M    Total params
111.311   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ConvNeXt         | 27.8 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
27.8 M    Trainable params
0         Non-trainable params
27.8 M    Total params
111.311   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ConvNeXt         | 27.8 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
27.8 M    Trainable params
0         Non-trainable params
27.8 M    Total params
111.311   Total estimated model params size (MB)


...

100%|██████████| 5000/5000 [00:43<00:00, 115.05it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0.   0.   0. 100.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       1.00      0.10      0.18      5000
         

100%|██████████| 5000/5000 [00:46<00:00, 108.55it/s]

Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0.   0.   0. 100.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       1.00      0.10      0.18      5000
         


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
detect(mnist_test_loader,tmodels[0].eval())
detect(cmnist_test_loader,tmodels[1].eval())
detect(svhn_test_loader,tmodels[2].eval())
del model, tmodels

100%|██████████| 5000/5000 [00:47<00:00, 105.02it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0. 100.   0.   0.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       1.00      0.10      0.18      5000
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

100%|██████████| 5000/5000 [00:42<00:00, 117.38it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0.   0.   0.   0. 100.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

100%|██████████| 5000/5000 [00:43<00:00, 114.48it/s]

Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0.   0.   0.   0. 100.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Regnet_y_400mf

In [ ]:
model, tmodels = federated_learning(client_datasets, num_epochs=20, name='regnet_y_400mf')
detect(mnist_test_loader,model.eval())
detect(cmnist_test_loader,model.eval())
detect(svhn_test_loader,model.eval())

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | RegNet           | 3.9 M 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
3.9 M     Trainable params
0         Non-trainable params
3.9 M     Total params
15.630    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | RegNet           | 3.9 M 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
3.9 M     Trainable params
0         Non-trainable params
3.9 M     Total params
15.630    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | RegNet           | 3.9 M 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
3.9 M     Trainable params
0         Non-trainable params
3.9 M     Total params
15.630    Total estimated model params size (MB)


...

100%|██████████| 5000/5000 [01:06<00:00, 75.63it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:

[[244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[100.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       1.00      0.10      0.18      5000
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

100%|██████████| 5000/5000 [01:05<00:00, 76.81it/s]


Confusion Matrix:

[[244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[100.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       1.00      0.10      0.18      5000
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
detect(mnist_test_loader,tmodels[0].eval())
detect(cmnist_test_loader,tmodels[1].eval())
detect(svhn_test_loader,tmodels[2].eval())
del model, tmodels

100%|██████████| 5000/5000 [01:03<00:00, 78.16it/s]


Confusion Matrix:

[[235   0   0   2   0   1   8   0  18   4]
 [  0 238   2   1   0   0   4   2  20   5]
 [  4   1 239   4   1   1   0   9  45   1]
 [  0   1   0 221   0  11   0   0   2   1]
 [  0   0   0   0 209   0   1   1   6  11]
 [  0   0   0   2   0 223   2   0   3   4]
 [  4   4   2   0  26   6 229   0   3   1]
 [  0   0   1  13   0   1   0 222   4   6]
 [  1   0   0   0   0   1   0   1 132   0]
 [  0   0   0   1   8   0   0   9  11 211]]

Overall Accuracy =  94.38 %

Class-Wise Accuracies =  [[98.2 98.8 99.  95.4 93.  95.8 97.  95.6 77.6 93.4]]
              precision    recall  f1-score   support

           0       0.98      0.94      0.96       524
           1       0.99      0.94      0.96       528
           2       0.99      0.88      0.93       561
           3       0.95      0.97      0.96       492
           4       0.93      0.96      0.95       484
           5       0.96      0.98      0.97       490
           6       0.97      0.91      0.94       531
        

100%|██████████| 5000/5000 [01:07<00:00, 74.35it/s]


Confusion Matrix:

[[ 73   0   1   0   1   0   9   0   5   1]
 [  0 234   3   0   3   0   4   4   1   5]
 [  2   6 228   4   2   0   0  10   6   2]
 [  0   0   1 218   0   4   0   0   1   3]
 [  1   1   0   0 228   0   2   1   3  14]
 [124   0   0  14   0 233   7   0   0   6]
 [  5   0   4   0   5   0 214   0   1   0]
 [  0   1   4   5   1   3   0 226   2  18]
 [ 39   2   3   2   1   4   8   1 221   5]
 [  0   0   0   1   3   0   0   2   4 190]]

Overall Accuracy =  92.5 %

Class-Wise Accuracies =  [[65.8 98.  96.8 94.8 96.8 97.8 94.  96.4 95.4 89.2]]
              precision    recall  f1-score   support

           0       0.66      0.95      0.78       346
           1       0.98      0.96      0.97       510
           2       0.97      0.94      0.95       516
           3       0.95      0.98      0.96       483
           4       0.97      0.96      0.96       506
           5       0.98      0.76      0.86       640
           6       0.94      0.97      0.95       485
         

100%|██████████| 5000/5000 [01:08<00:00, 72.60it/s]

Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   1   0]
 [  0   0   0   0   0   0   1   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 243 244 244 244 244 243 244 243 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   1   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0.   0. 100.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         0
           5       1.00      0.10      0.18      4997
           6       0.00      0.00      0.00         0
         


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Resnet_34

In [ ]:
model, tmodels = federated_learning(client_datasets, num_epochs=20, name='resnet_34')
detect(mnist_test_loader,model.eval())
detect(cmnist_test_loader,model.eval())
detect(svhn_test_loader,model.eval())

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 21.3 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
21.3 M    Trainable params
0         Non-trainable params
21.3 M    Total params
85.159    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 21.3 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
21.3 M    Trainable params
0         Non-trainable params
21.3 M    Total params
85.159    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 21.3 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
21.3 M    Trainable params
0         Non-trainable params
21.3 M    Total params
85.159    Total estimated model params size (MB)


...

100%|██████████| 5000/5000 [00:30<00:00, 163.51it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:

[[244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[100.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       1.00      0.10      0.18      5000
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

100%|██████████| 5000/5000 [00:31<00:00, 160.67it/s]

Confusion Matrix:

[[244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[100.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       1.00      0.10      0.18      5000
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
detect(mnist_test_loader,tmodels[0].eval())
detect(cmnist_test_loader,tmodels[1].eval())
detect(svhn_test_loader,tmodels[2].eval())
del model, tmodels

100%|██████████| 5000/5000 [00:30<00:00, 161.45it/s]


Confusion Matrix:

[[239   0   2   0   1   0   6   0   2   2]
 [  0 231   7   0   0   0   2   0   0   2]
 [  0   0   6   0   0   0   0   0   0   0]
 [  1   2  14 238   0   6   0   4  12  18]
 [  0   0   2   0 237   0   2   0   0  47]
 [  2   3   0   5   0 237  11   2   1   7]
 [  2   0   0   0   2   0 217   0   0   0]
 [  0   8 209   1   2   0   0 238   6   5]
 [  0   0   4   0   1   1   6   0 222   3]
 [  0   0   0   0   1   0   0   0   1 160]]

Overall Accuracy =  91.7 %

Class-Wise Accuracies =  [[99.  97.4 52.4 98.8 98.6 98.6 94.6 98.8 95.6 83.2]]
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       508
           1       0.97      0.98      0.98       498
           2       0.52      1.00      0.69       262
           3       0.99      0.90      0.94       551
           4       0.99      0.91      0.94       544
           5       0.99      0.94      0.96       524
           6       0.95      0.99      0.97       477
         

100%|██████████| 5000/5000 [00:31<00:00, 159.06it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:

[[182 140 200 173 166 170 178 164 177 178]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [ 62 104  44  71  78  74  66  80  67  66]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.24 %

Class-Wise Accuracies =  [[87.6  0.   0.   0.   0.  14.8  0.   0.   0.   0. ]]
              precision    recall  f1-score   support

           0       0.88      0.10      0.18      4288
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.15      0.10      0.12       712
           6       0.00      0.00      0.00         0
        

100%|██████████| 5000/5000 [00:30<00:00, 162.92it/s]

Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  1   2   0   3   0   1   1   0   1   1]
 [  5  13   8  13  12   9  14  14  12  17]
 [ 89  78  91  89  86  91  78 101  79  78]
 [  0   0   0   0   0   0   0   0   0   0]
 [  3   6   6   4   4   7   4   2  10   8]
 [  0   0   0   0   0   0   0   0   0   0]
 [ 39  39  39  51  58  31  46  35  53  53]
 [  0   0   0   0   0   0   0   0   0   0]
 [107 106 100  84  84 105 101  92  89  87]]

Overall Accuracy =  9.68 %

Class-Wise Accuracies =  [[ 0.   0.4  1.6 17.8  0.   1.4  0.  58.2  0.  17.4]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.20      0.01        10
           2       0.02      0.07      0.03       117
           3       0.18      0.10      0.13       860
           4       0.00      0.00      0.00         0
           5       0.01      0.13      0.03        54
           6       0.00      0.00      0.00         0
         


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Using Normal Method

## mobileNet_v3_large

In [ ]:
client_model_1 = Model('mobilenet_v3_large',10)
trainer = pl.Trainer(accelerator='gpu', max_epochs=10, default_root_dir='/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/mobilenet_v3_large_1', benchmark=True)
trainer.fit(client_model_1, mnist_train_loader)
del client_model_1, trainer

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | MobileNetV3      | 4.2 M 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
4.2 M     Trainable params
0         Non-trainable params
4.2 M     Total params
16.859    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
client_model_2 = Model('mobilenet_v3_large',10)
trainer = pl.Trainer(accelerator='gpu', max_epochs=10, default_root_dir='/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/mobilenet_v3_large_2', benchmark=True)
trainer.fit(client_model_2, cmnist_train_loader)
del client_model_2, trainer

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | MobileNetV3      | 4.2 M 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
4.2 M     Trainable params
0         Non-trainable params
4.2 M     Total params
16.859    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
client_model_3 = Model('mobilenet_v3_large',10)
trainer = pl.Trainer(accelerator='gpu', max_epochs=10, default_root_dir='/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/mobilenet_v3_large_3', benchmark=True)
trainer.fit(client_model_3, cmnist_train_loader)
del client_model_3, trainer

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | MobileNetV3      | 4.2 M 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
4.2 M     Trainable params
0         Non-trainable params
4.2 M     Total params
16.859    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
client_model_1 = model_return('/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/mobilenet_v3_large_1/lightning_logs/version_0/checkpoints/epoch=9-step=25000.ckpt', Model('mobilenet_v3_large',10))
client_model_2 = model_return('/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/mobilenet_v3_large_2/lightning_logs/version_0/checkpoints/epoch=9-step=25000.ckpt', Model('mobilenet_v3_large',10))
client_model_3 = model_return('/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/mobilenet_v3_large_3/lightning_logs/version_0/checkpoints/epoch=9-step=25000.ckpt', Model('mobilenet_v3_large',10))

In [ ]:
detect(mnist_test_loader, client_model_1)

100%|██████████| 5000/5000 [00:47<00:00, 104.88it/s]


Confusion Matrix:

[[239   0   1   1   0   1   6   0   2   1]
 [  0 238   3   0   1   0   4   0   3   1]
 [  0   3 215   8   0   0   0   4   3   0]
 [  0   0   3 205   0   3   0   0   1   0]
 [  0   0   4   0 226   0  11   0   0   6]
 [  3   1   0   6   0 238  22   0   1   3]
 [  1   0   0   0   0   0 197   0   0   0]
 [  0   1  13  22   0   1   0 233   4   7]
 [  1   1   5   1   0   1   4   0 225   2]
 [  0   0   0   1  17   0   0   7   5 224]]

Overall Accuracy =  96.0 %

Class-Wise Accuracies =  [[99.  98.8 94.2 92.2 96.4 98.8 90.6 97.8 96.2 96. ]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       507
           1       0.99      0.98      0.98       506
           2       0.94      0.96      0.95       489
           3       0.92      0.99      0.95       468
           4       0.96      0.96      0.96       503
           5       0.99      0.93      0.96       530
           6       0.91      1.00      0.95       454
         

In [ ]:
detect(mnist_test_loader, client_model_2)

100%|██████████| 5000/5000 [00:47<00:00, 105.97it/s]

Confusion Matrix:

[[201   0   3   0   0   0   8   1   4   1]
 [  5 241  51  16  63  10  34 145  46  65]
 [  0   0 161   0   0   0   0   4   0   0]
 [  1   1  14 206   0  26   0   0   0   3]
 [  0   0   2   0 174   0   1   3   0  24]
 [ 16   0   1   4   1 175  14   0   0   1]
 [  2   0   1   0   4   0 108   0   0   0]
 [  0   0   0   1   0   0   0  87   0   0]
 [ 19   2  11  17   2  33  79   0 192   2]
 [  0   0   0   0   0   0   0   4   2 148]]

Overall Accuracy =  85.06 %

Class-Wise Accuracies =  [[91.4 99.4 83.4 92.4 86.  86.2 72.8 68.6 89.6 80.8]]
              precision    recall  f1-score   support

           0       0.91      0.96      0.94       474
           1       0.99      0.53      0.69       932
           2       0.83      0.99      0.91       421
           3       0.92      0.91      0.92       507
           4       0.86      0.93      0.90       460
           5       0.86      0.92      0.89       468
           6       0.73      0.98      0.84       371
        

In [ ]:
detect(svhn_test_loader, client_model_3)

100%|██████████| 5000/5000 [00:47<00:00, 105.24it/s]

Confusion Matrix:

[[  6   1   6   1   3   2   2   2   1   2]
 [ 14 107  23   6   8   1   4  72   4   5]
 [  3   1  25   9   3   3   3  18   7   3]
 [  0   0   0   7   0   1   0   0   0   0]
 [  0   0   0   1   5   0   0   0   1  13]
 [147  32  56 172 178 227 231  28 183 151]
 [ 11  12   8   3  13   2  16   5   4   1]
 [  5   6   7   4   5  31   8  41   4  15]
 [ 58  85 119  41  29 231 235  78  40  46]
 [  0   0   0   0   0   2   1   0   0   8]]

Overall Accuracy =  14.76 %

Class-Wise Accuracies =  [[ 1.2 21.4  5.   1.4  1.  45.4  3.2  8.2 59.2  1.6]]
              precision    recall  f1-score   support

           0       0.01      0.23      0.02        26
           1       0.21      0.44      0.29       244
           2       0.05      0.33      0.09        75
           3       0.01      0.88      0.03         8
           4       0.01      0.25      0.02        20
           5       0.45      0.16      0.24      1405
           6       0.03      0.21      0.06        75
        

In [ ]:
server_model = Model('mobilenet_v3_large',10).eval().cuda()
for name, param in server_model.named_parameters():
  param.data = torch.zeros_like(param.data)
  for client_model in [client_model_1,client_model_2,client_model_3]:
    param.data += client_model.state_dict()[name] / 3

In [ ]:
detect(mnist_test_loader, server_model)

100%|██████████| 5000/5000 [00:47<00:00, 106.07it/s]


Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0.   0. 100.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       1.00      0.10      0.18      5000
           6       0.00      0.00      0.00         0
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
detect(cmnist_test_loader, server_model)

100%|██████████| 5000/5000 [00:49<00:00, 101.73it/s]

Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0.   0. 100.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       1.00      0.10      0.18      5000
           6       0.00      0.00      0.00         0
         


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
detect(svhn_test_loader, server_model)

100%|██████████| 5000/5000 [00:47<00:00, 105.66it/s]


Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0.   0. 100.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       1.00      0.10      0.18      5000
           6       0.00      0.00      0.00         0
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
client_models = [client_model_1, client_model_2, client_model_3]
server_model = Model('mobilenet_v3_large',10).eval().cuda()
server_model = server_aggregate(server_model, client_models)
detect(mnist_test_loader, server_model)

100%|██████████| 5000/5000 [00:47<00:00, 105.95it/s]


Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0.   0. 100.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       1.00      0.10      0.18      5000
           6       0.00      0.00      0.00         0
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
del client_model_1, client_model_2, client_model_3, server_model

In [ ]:
combined_model_1 = Model('mobilenet_v3_large',10)
trainer = pl.Trainer(accelerator='gpu', max_epochs=10, default_root_dir='/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/mobilenet_v3_large_combined', benchmark=True)
trainer.fit(combined_model_1, train_dataloader)
detect(test_dataloader, combined_model_1.eval())
del combined_model_1, trainer


0it [00:00, ?it/s]
3it [00:00, 26.49it/s]
10it [00:00, 49.59it/s]
20it [00:00, 71.02it/s]
32it [00:00, 87.36it/s]
43it [00:00, 93.87it/s]
54it [00:00, 97.27it/s]
65it [00:00, 100.70it/s]
76it [00:00, 100.77it/s]
87it [00:00, 98.14it/s] 
98it [00:01, 101.15it/s]
109it [00:01, 102.46it/s]
120it [00:01, 102.97it/s]
131it [00:01, 100.99it/s]
142it [00:01, 99.41it/s] 
152it [00:01, 99.34it/s]
162it [00:01, 99.35it/s]
173it [00:01, 99.66it/s]
184it [00:01, 100.17it/s]
195it [00:02, 101.90it/s]
206it [00:02, 100.81it/s]
217it [00:02, 100.34it/s]
228it [00:02, 101.26it/s]
239it [00:02, 102.45it/s]
250it [00:02, 103.05it/s]
261it [00:02, 101.98it/s]
272it [00:02, 101.78it/s]
283it [00:02, 101.51it/s]
294it [00:02, 103.55it/s]
305it [00:03, 104.55it/s]
317it [00:03, 106.55it/s]
329it [00:03, 108.59it/s]
341it [00:03, 109.32it/s]
352it [00:03, 106.04it/s]
363it [00:03, 104.32it/s]
374it [00:03, 104.66it/s]
386it [00:03, 106.99it/s]
398it [00:03, 108.38it/s]
409it [00:04, 108.14it/s]
420it [00:04

Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [220 220 220 220 220 220 220 220 220 220]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0.   0.   0.   0.   0.   0. 100.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## swin_t

In [ ]:
client_model_1 = Model('swin_t',10)
trainer = pl.Trainer(accelerator='gpu', max_epochs=10, default_root_dir='/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/swin_t_1', benchmark=True)
trainer.fit(client_model_1, mnist_train_loader)
del client_model_1, trainer

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | SwinTransformer  | 27.5 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
27.5 M    Trainable params
0         Non-trainable params
27.5 M    Total params
110.108   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
client_model_2 = Model('swin_t',10)
trainer = pl.Trainer(accelerator='gpu', max_epochs=10, default_root_dir='/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/swin_t_2', benchmark=True)
trainer.fit(client_model_2, cmnist_train_loader)
del client_model_2, trainer

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | SwinTransformer  | 27.5 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
27.5 M    Trainable params
0         Non-trainable params
27.5 M    Total params
110.108   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
client_model_3 = Model('swin_t',10)
trainer = pl.Trainer(accelerator='gpu', max_epochs=10, default_root_dir='/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/swin_t_3', benchmark=True)
trainer.fit(client_model_3, cmnist_train_loader)
del client_model_3, trainer

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | SwinTransformer  | 27.5 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
27.5 M    Trainable params
0         Non-trainable params
27.5 M    Total params
110.108   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
client_model_1 = model_return('/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/swin_t_1/lightning_logs/version_0/checkpoints/epoch=9-step=25000.ckpt', Model('swin_t',10))
client_model_2 = model_return('/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/swin_t_2/lightning_logs/version_0/checkpoints/epoch=9-step=25000.ckpt', Model('swin_t',10))
client_model_3 = model_return('/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/swin_t_3/lightning_logs/version_0/checkpoints/epoch=9-step=25000.ckpt', Model('swin_t',10))

In [ ]:
detect(mnist_test_loader, client_model_1)

100%|██████████| 5000/5000 [01:23<00:00, 60.01it/s]

Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0.   0.   0.   0. 100.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
detect(mnist_test_loader, client_model_2)

100%|██████████| 5000/5000 [01:21<00:00, 61.64it/s]


Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0.   0. 100.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       1.00      0.10      0.18      5000
           6       0.00      0.00      0.00         0
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
detect(svhn_test_loader, client_model_3)

100%|██████████| 5000/5000 [01:20<00:00, 62.48it/s]

Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0. 100.   0.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       1.00      0.10      0.18      5000
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
server_model = Model('swin_t',10).eval().cuda()
for name, param in server_model.named_parameters():
  param.data = torch.zeros_like(param.data)
  for client_model in [client_model_1,client_model_2,client_model_3]:
    param.data += client_model.state_dict()[name] / 3

In [ ]:
detect(mnist_test_loader, server_model)

100%|██████████| 5000/5000 [01:21<00:00, 61.39it/s]

Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0.   0.   0. 100.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       1.00      0.10      0.18      5000
         


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
detect(cmnist_test_loader, server_model)

100%|██████████| 5000/5000 [01:22<00:00, 60.84it/s]

Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0.   0.   0. 100.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       1.00      0.10      0.18      5000
         


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
detect(svhn_test_loader, server_model)

100%|██████████| 5000/5000 [01:20<00:00, 61.75it/s]

Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0.   0.   0. 100.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       1.00      0.10      0.18      5000
         


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
del client_model_1, client_model_2, client_model_3, server_model

In [ ]:
combined_model_1 = Model('swin_t',10)
trainer = pl.Trainer(accelerator='gpu', max_epochs=10, default_root_dir='/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/swin_t_combined', benchmark=True)
trainer.fit(combined_model_1, train_dataloader)
detect(test_dataloader, combined_model_1.eval())
del combined_model_1, trainer

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | SwinTransformer  | 27.5 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
27.5 M    Trainable params
0         Non-trainable params
27.5 M    Total params
110.108   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.

0it [00:00, ?it/s]
5it [00:00, 46.45it/s]
11it [00:00, 52.04it/s]
17it [00:00, 53.64it/s]
23it [00:00, 55.58it/s]
29it [00:00, 54.67it/s]
35it [00:00, 55.31it/s]
41it [00:00, 55.44it/s]
47it [00:00, 56.13it/s]
53it [00:00, 56.29it/s]
59it [00:01, 56.74it/s]
65it [00:01, 56.75it/s]
71it [00:01, 54.87it/s]
77it [00:01, 54.72it/s]
83it [00:01, 53.51it/s]
89it [00:01, 53.19it/s]
95it [00:01, 53.49it/s]
101it [00:01, 54.16it/s]
107it [00:01, 54.70it/s]
113it [00:02, 55.29it/s]
119it [00:02, 55.24it/s]
125it [00:02, 55.10it/s]
131it [00:02, 49.38it/s]
137it [00:02, 45.32it/s]
142it [00:02, 44.11it/s]
147it [00:02, 43.67it/s]
152it [00:02, 42.30it/s]
157it [00:03, 42.03it/s]
162it [00:03, 41.66it/s]
167it [00:03, 40.76it/s]
172it [00:03, 41.28it/s]
177it [00:03, 39.99it/s]
182it [00:03, 40.32it/s]
187it [00:03, 39.89it/s]
191it [00:03, 38.69it/s]
195it [00:04, 38.82it/s]
199it [00:04, 38.90it/s]
203it 

Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [220 220 220 220 220 220 220 220 220 220]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0.   0.   0.   0.   0.   0.   0.   0.   0. 100.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## densenet_121

In [ ]:
client_model_1 = Model('densenet_121',10)
trainer = pl.Trainer(accelerator='gpu', max_epochs=10, default_root_dir='/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/densenet_121_1', benchmark=True)
trainer.fit(client_model_1, mnist_train_loader)
del client_model_1, trainer

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | DenseNet         | 7.0 M 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
7.0 M     Trainable params
0         Non-trainable params
7.0 M     Total params
27.856    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
client_model_2 = Model('densenet_121',10)
trainer = pl.Trainer(accelerator='gpu', max_epochs=10, default_root_dir='/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/densenet_121_2', benchmark=True)
trainer.fit(client_model_2, cmnist_train_loader)
del client_model_2, trainer

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | DenseNet         | 7.0 M 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
7.0 M     Trainable params
0         Non-trainable params
7.0 M     Total params
27.856    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
client_model_3 = Model('densenet_121',10)
trainer = pl.Trainer(accelerator='gpu', max_epochs=10, default_root_dir='/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/densenet_121_3', benchmark=True)
trainer.fit(client_model_3, cmnist_train_loader)
del client_model_3, trainer

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | DenseNet         | 7.0 M 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
7.0 M     Trainable params
0         Non-trainable params
7.0 M     Total params
27.856    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
client_model_1 = model_return('/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/densenet_121_1/lightning_logs/version_0/checkpoints/epoch=9-step=25000.ckpt', Model('densenet_121',10))
client_model_2 = model_return('/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/densenet_121_2/lightning_logs/version_0/checkpoints/epoch=9-step=25000.ckpt', Model('densenet_121',10))
client_model_3 = model_return('/content/drive/MyDrive/DAI_Assignment_3/Normal_Method/densenet_121_3/lightning_logs/version_1/checkpoints/epoch=9-step=25000.ckpt', Model('densenet_121',10))

In [ ]:
detect(mnist_test_loader, client_model_1)

100%|██████████| 5000/5000 [01:51<00:00, 44.73it/s]

Confusion Matrix:

[[221   0   0   0   0   0   3   0   0   0]
 [  1 123   0   0   0   0   0   5   0   0]
 [  0  31 244   4   2   1   1   7   2   0]
 [  3   2   0 238   0   6   0   0   1   3]
 [  5  87   0   0 239   1   6   2   1  12]
 [  0   0   0   0   0 226   0   0   0   5]
 [ 10   0   0   0   0   5 231   0   0   0]
 [  0   1   0   2   0   1   0 225   2   1]
 [  0   0   0   0   0   4   3   0 231   1]
 [  4   0   0   0   3   0   0   5   7 222]]

Overall Accuracy =  95.2 %

Class-Wise Accuracies =  [[ 95.4  75.8 100.   98.8  99.   96.4  97.4  96.2  97.4  95.6]]
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       480
           1       0.76      0.98      0.86       385
           2       1.00      0.91      0.95       548
           3       0.99      0.97      0.98       509
           4       0.99      0.81      0.89       609
           5       0.96      0.99      0.98       487
           6       0.97      0.97      0.97       502

In [ ]:
detect(mnist_test_loader, client_model_2)

100%|██████████| 5000/5000 [01:50<00:00, 45.12it/s]

Confusion Matrix:

[[125   0   0   0   0   0   0   0   0   0]
 [  3 239  12   2   0   0   4   9   3   3]
 [  4   0 216   5   0   0   2   3  12   5]
 [  0   0   0 220   0   0   0   0   3   0]
 [  4   0   4   0 244   0   5   3   0  44]
 [ 87   4   0  13   0 244  32   1  20   6]
 [  3   1   0   0   0   0 201   0   0   0]
 [ 13   0  12   4   0   0   0 228   5   6]
 [  0   0   0   0   0   0   0   0 200   0]
 [  5   0   0   0   0   0   0   0   1 180]]

Overall Accuracy =  88.02 %

Class-Wise Accuracies =  [[ 25.   99.   94.4  95.2 100.  100.   91.4  96.8  91.2  87.2]]
              precision    recall  f1-score   support

           0       0.25      1.00      0.40       125
           1       0.99      0.93      0.96       531
           2       0.94      0.94      0.94       503
           3       0.95      0.99      0.97       479
           4       1.00      0.89      0.94       560
           5       1.00      0.54      0.70       919
           6       0.91      0.99      0.95       46

In [ ]:
detect(svhn_test_loader, client_model_3)

100%|██████████| 5000/5000 [01:51<00:00, 44.93it/s]

Confusion Matrix:

[[ 39   1   5  26   8  17  40   1  25  31]
 [191 235 223 186 198 196 188 236 171 155]
 [  0   0   1   0   0   0   0   0   0   0]
 [  0   0   0  15   0   1   0   0   2   0]
 [ 14   6   9  11  37   9   4   1   6  45]
 [  0   0   0   0   0  12   0   0   0   0]
 [  0   1   0   0   0   2   7   0   0   1]
 [  0   1   5   2   1   5   0   6   3   5]
 [  0   0   1   4   0   2   4   0  37   4]
 [  0   0   0   0   0   0   1   0   0   3]]

Overall Accuracy =  12.96 %

Class-Wise Accuracies =  [[ 7.8 98.2  0.2  3.   7.4  2.4  1.4  1.2  7.4  0.6]]
              precision    recall  f1-score   support

           0       0.08      0.20      0.11       193
           1       0.98      0.11      0.19      4539
           2       0.00      1.00      0.00         1
           3       0.03      0.83      0.06        18
           4       0.07      0.26      0.12       142
           5       0.02      1.00      0.05        12
           6       0.01      0.64      0.03        11
        

In [ ]:
server_model = Model('densenet_121',10).eval().cuda()
for name, param in server_model.named_parameters():
  param.data = torch.zeros_like(param.data)
  for client_model in [client_model_1,client_model_2,client_model_3]:
    param.data += client_model.state_dict()[name] / 3

In [ ]:
detect(mnist_test_loader, server_model)

100%|██████████| 5000/5000 [01:51<00:00, 44.70it/s]

Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0. 100.   0.   0.   0.   0.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.10      0.18      5000
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
detect(cmnist_test_loader, server_model)

100%|██████████| 5000/5000 [01:55<00:00, 43.42it/s]


Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0. 100.   0.   0.   0.   0.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.10      0.18      5000
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
detect(svhn_test_loader, server_model)

100%|██████████| 5000/5000 [01:57<00:00, 42.58it/s]

Confusion Matrix:

[[  0   0   0   0   0   0   0   0   0   0]
 [244 244 244 244 244 244 244 244 244 244]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]

Overall Accuracy =  10.0 %

Class-Wise Accuracies =  [[  0. 100.   0.   0.   0.   0.   0.   0.   0.   0.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.10      0.18      5000
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
         


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
del client_model_1, client_model_2, client_model_3, server_model